# CSE475 Project Bonus, Due: Friday, 05/02/2025

## Instruction

1. Please submit your Jupyter Notebook file (the. ipynb file) containing your code and the outputs produced by your code (note that .ipynb file can contain both the code and the outputs) to Canvas. Please name your file CSE475-ProjectBonus-LastName-FirstName.ipynb.

2. If you have any questions on the homework problems, you should post your question on the Canvas discussion board (under Project Q&A), instead of sending emails to the instructor or TA. We will answer your questions there. In this way, we can avoid repeated questions, and help the entire class stay on the same page whenever any clarification/correction is made.

## Building a Convolutional Neural Network to classify images in the CIFAR-10 Dataset

We will work with the CIFAR-10 Dataset.  This is a well-known dataset for image classification, which consists of 60000 32x32 color images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The 10 classes are:

<ol start="0">
<li> airplane
<li>  automobile
<li> bird
<li>  cat
<li> deer
<li> dog
<li>  frog
<li>  horse
<li>  ship
<li>  truck
</ol>

For details about CIFAR-10 see:
https://www.cs.toronto.edu/~kriz/cifar.html

For a compilation of published performance results on CIFAR 10, see:
http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

---

### Building CNN

In this project we will build and train our convolutional neural network. In the first part, we walk through different layers and how they are configured. In the second part, you will build your own model, train it, and compare the performance.

In [3]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
## Each image is a 32 x 32 x 3 numpy array
x_train[444].shape

(32, 32, 3)

In [6]:
## Let's look at one of the images

print(y_train[444])
plt.imshow(x_train[444])

[9]


In [7]:
# convert class labels to one-hot vectors
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
# see some one-hot vector
y_train[444]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [9]:
# As before, let's make everything float and scale
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## First CNN
Below we will build our first CNN.  For demonstration purpose (so that it will br trained quickly) it is not very deep and has relatively few parameters.  We use strides of 2 in the first two convolutional layers which quickly reduces the dimensions of the output. After a MaxPooling layer, we flatten, and then have a single fully connected layer before the final classification layer.

In [10]:
# Let's build a CNN using Keras' Sequential capabilities

model_1 = Sequential()

## 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## Another 5x5 convolution with 2x2 stride and 32 filters
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## 2x2 max pooling reduces to 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten turns 3x3x32 into 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 16, 16, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 6, 6, 32)       │        25,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 6, 6, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 3, 3, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3, 3, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 288)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       147,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 181,162 (707.66 KB)

 Trainable params: 181,162 (707.66 KB)

 Non-trainable params: 0 (0.00 B)

We still have 181K parameters, even though this is a "small" model.


In [12]:
# batch_size = 128

# # initiate RMSprop optimizer
# opt = keras.optimizers.rmsprop(lr=0.0005, decay=1e-6)

# # Let's train the model using RMSprop
# model_1.compile(loss='categorical_crossentropy',
#               optimizer=opt,
#               metrics=['accuracy'])

# model_1.fit(x_train, y_train,
#               batch_size=batch_size,
#               epochs=15,
#               validation_data=(x_test, y_test),
#               shuffle=True)

from tensorflow.keras.optimizers import RMSprop

batch_size = 128

# Initiate RMSprop optimizer with updated parameters
opt = RMSprop(learning_rate=0.0005, decay=1e-6)

# Compile the model with categorical crossentropy and accuracy metrics
model_1.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Train the model
model_1.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=15,
    validation_data=(x_test, y_test),
    shuffle=True
)


/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.2341 - loss: 2.0628 - val_accuracy: 0.4299 - val_loss: 1.6099
Epoch 2/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4033 - loss: 1.6501 - val_accuracy: 0.4788 - val_loss: 1.4621
Epoch 3/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.4537 - loss: 1.5167 - val_accuracy: 0.4650 - val_loss: 1.5170
Epoch 4/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4847 - loss: 1.4281 - val_accuracy: 0.5429 - val_loss: 1.2974
Epoch 5/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5061 - loss: 1.3709 - val_accuracy: 0.5631 - val_loss: 1.2373
Epoch 6/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5327 - loss: 1.3156 - val_accuracy: 0.5524 - val_loss: 1.2696
Epoch 7/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5501 - loss: 1.2594 - val_accuracy: 0.5817 - val_loss: 1.1795
Epoch 8/15
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5575 - loss: 1.2401 - val_accu

In [13]:
score = model_1.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.0074163675308228
Test accuracy: 0.6471999883651733


## Your task (25pts)

Our previous model (model_1) had the structure:

Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification (with activation functions and dropouts)

Please built a different model (named model_2) by trying different structures and different hyperparameters, such as number of neurons, layers, stride, padding, dropout rate, kernel size, learning rate, number of epochs, etc. You can choose to add data augmentation, batch normalization and/or something new.<br>

For example: <br>
A deeper model: Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Final Classification
<br>

Report the best test accuracy achieved. You will be graded on the highest test accuracy achieved:<br>
Test accuracy < Base model (model_1) : 0 - 5pts (Depending on the changes made in model_2)<br>
Base model (model_1) < Test accuracy < 70%: 5 - 10pts (Depending on the changes made in model_2)<br>
70% < Test accuracy < 75%: 15pts<br>
75% < Test accuracy: 25pts <br>

In [14]:
# Let's build a CNN using Keras' Sequential capabilities
model_2 = Sequential()

In [16]:
#write your code here

from tensorflow.keras.layers import Flatten,LeakyReLU
# 1st Conv + Pool
model_2.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model_2.add(LeakyReLU(alpha=0.1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

# 2nd Conv + Pool
model_2.add(Conv2D(64, (3, 3), padding='same'))
model_2.add(LeakyReLU(alpha=0.1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

# 3rd Conv + Pool
model_2.add(Conv2D(128, (3, 3), padding='same'))
model_2.add(LeakyReLU(alpha=0.1))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and Fully Connected Layers
model_2.add(Flatten())

model_2.add(Dense(256))
model_2.add(LeakyReLU(alpha=0.1))

model_2.add(Dense(128))
model_2.add(LeakyReLU(alpha=0.1))

model_2.add(Dense(num_classes))
model_2.add(Activation('softmax'))

/Users/atharva7/miniconda3/lib/python3.10/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [18]:
from tensorflow.keras.optimizers import RMSprop

opt2 = RMSprop(learning_rate=0.0005, decay=1e-6)
model_2.compile(loss='categorical_crossentropy',
                optimizer=opt2,
                metrics=['accuracy'])

In [19]:
history_2 = model_2.fit(x_train, y_train,
                        batch_size=128,
                        epochs=30,
                        validation_data=(x_test, y_test),
                        shuffle=True)


Epoch 1/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 34s 86ms/step - accuracy: 0.2921 - loss: 1.9282 - val_accuracy: 0.5067 - val_loss: 1.3538
Epoch 2/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 36s 93ms/step - accuracy: 0.5420 - loss: 1.2896 - val_accuracy: 0.6075 - val_loss: 1.0918
Epoch 3/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 36s 92ms/step - accuracy: 0.6519 - loss: 0.9912 - val_accuracy: 0.6800 - val_loss: 0.9282
Epoch 4/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 38s 97ms/step - accuracy: 0.7183 - loss: 0.8063 - val_accuracy: 0.7135 - val_loss: 0.8327
Epoch 5/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 45s 114ms/step - accuracy: 0.7666 - loss: 0.6724 - val_accuracy: 0.6981 - val_loss: 0.8964
Epoch 6/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 36s 92ms/step - accuracy: 0.7981 - loss: 0.5811 - val_accuracy: 0.7388 - val_loss: 0.7940
Epoch 7/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 101ms/step - accuracy: 0.8374 - loss: 0.4709 - val_accuracy: 0.7503 - val_loss: 0.7533
Epoch 8/30
391/391 ━━━━━━━━━━━━━━━━━━━━ 43s 111ms/step - accuracy: 0.8613 - loss: 0.3932

In [20]:
# Test the model on test data
score = model_2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.1156017780303955
Test accuracy: 0.7595999836921692
